In [15]:
import pandas as pd
import sqlite3 as sq
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

In [29]:
conn = sq.connect('../../Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [WPURBANSIM_Employment_CurrentandProjected_Annual_WideFormat_2023Base]', conn)
initial = pd.DataFrame(sql_query)
initial = initial[['NAME', 'Employment 2017', 'Employment 2020', 'Employment 2025', 'Employment 2035', 'Employment 2045']]
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
           'Humphreys County, Tennessee', 'Maury County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 
           'Rutherford County, Tennessee', 'Stewart County, Tennessee', 'Sumner County, Tennessee', 'Williamson County, Tennessee', 
           'Wilson County, Tennessee', 'Trousdale County, Tennessee', 'GNRC', 'MPO', 'GNRC Region']
initial = initial.loc[initial['NAME'].isin(thelist)].reset_index(drop = True)
initial = initial.rename(columns = {'Employment 2017': '2017', 'Employment 2020': '2020', 'Employment 2025': '2025', 
                                      'Employment 2035': '2035', 'Employment 2045': '2045'})

#first option
#initial['Base'] = '2023'
# second option
initial = initial.melt(id_vars = ['NAME'], var_name = 'Year', value_name = 'Employment 2023 Base')

In [31]:
initial.tail()

,NAME,Year,Employment 2023 Base
80,"Stewart County, Tennessee",2045,5778.0
81,"Sumner County, Tennessee",2045,155391.0
82,"Trousdale County, Tennessee",2045,4380.0
83,"Williamson County, Tennessee",2045,480779.0
84,"Wilson County, Tennessee",2045,142835.0


In [32]:
conn = sq.connect('../../Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [WPURBANSIM_Employment_CurrentandProjected_Annual_WideFormat_2017Base]', conn)
initial1 = pd.DataFrame(sql_query)
initial1 = initial1[['NAME', 'Employment 2017', 'Employment 2020', 'Employment 2025', 'Employment 2035', 'Employment 2045']]
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
           'Humphreys County, Tennessee', 'Maury County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 
           'Rutherford County, Tennessee', 'Stewart County, Tennessee', 'Sumner County, Tennessee', 'Williamson County, Tennessee', 
           'Wilson County, Tennessee', 'Trousdale County, Tennessee', 'GNRC', 'MPO', 'GNRC Region']
initial1['NAME'] = initial1['NAME'].rename({'GNRC Region': 'GNRCRegion'})
initial1 = initial1.loc[initial1['NAME'].isin(thelist)].reset_index(drop = True)
initial1 = initial1.rename(columns = {'Employment 2017': '2017', 'Employment 2020': '2020', 'Employment 2025': '2025', 
                                      'Employment 2035': '2035', 'Employment 2045': '2045'})

#first option
#initial1['Base'] = '2017'
# # second option
initial1 = initial1.melt(id_vars = ['NAME'], var_name = 'Year', value_name = 'Employment 2017 Base')

In [33]:
initial1.head()

,NAME,Year,Employment 2017 Base
0,"Cheatham County, Tennessee",2017,14923.0
1,"Davidson County, Tennessee",2017,619069.0
2,"Dickson County, Tennessee",2017,25227.0
3,GNRC,2017,1283852.0
4,GNRC Region,2017,1331775.0


In [34]:
conn = sq.connect('../../Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [BEA2017_2022Employment]', conn)
bea = pd.DataFrame(sql_query)
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 
           'Humphreys County, Tennessee', 'Maury County, Tennessee', 'Montgomery County, Tennessee', 'Robertson County, Tennessee', 
           'Rutherford County, Tennessee', 'Stewart County, Tennessee', 'Sumner County, Tennessee', 'Williamson County, Tennessee', 
           'Wilson County, Tennessee', 'Trousdale County, Tennessee', 'GNRC', 'MPO', 'GNRC Region']
bea = bea.loc[bea['NAME'].isin(thelist)]
bea = bea.loc[(bea['Sub Category'] == 'Total')]
bea = bea[['NAME', '2017', '2018', '2019', '2020', '2021', '2022']]

#first option
#initial['Base'] = 'BEA'
# second option
bea = bea.melt(id_vars = ['NAME'], var_name = 'Year', value_name = 'Employment BEA')

In [35]:
bea.head()

,NAME,Year,Employment BEA
0,"Cheatham County, Tennessee",2017,15234.0
1,"Davidson County, Tennessee",2017,644850.0
2,"Dickson County, Tennessee",2017,25766.0
3,"Houston County, Tennessee",2017,2698.0
4,"Humphreys County, Tennessee",2017,8789.0


In [38]:
#data = initial.merge(initial1, on = ['NAME', 'Year'])
data = data.merge(bea, on = ['NAME', 'Year'], how = 'outer')
data.head()

,NAME,Year,Employment 2023 Base,Employment 2017 Base,Employment BEA
0,"Cheatham County, Tennessee",2017,15234.0,14923.0,15234.0
1,"Davidson County, Tennessee",2017,644843.0,619069.0,644850.0
2,"Dickson County, Tennessee",2017,25762.0,25227.0,25766.0
3,GNRC,2017,1344827.0,1283852.0,1344819.0
4,GNRC Region,2017,1395681.0,1331775.0,1395674.0


In [39]:
data.tail()

,NAME,Year,Employment 2023 Base,Employment 2017 Base,Employment BEA
148,"Williamson County, Tennessee",2022,NaN,NaN,238165.0
149,"Wilson County, Tennessee",2022,NaN,NaN,94211.0
150,MPO,2022,NaN,NaN,1480965.0
151,GNRC,2022,NaN,NaN,1583744.0
152,GNRC Region,2022,NaN,NaN,1643234.0


In [41]:
data['Difference 2023-2017 Base'] = data['Employment 2023 Base'] - data['Employment 2017 Base']
data['Difference % 2023-2017 Base'] = ((data['Employment 2023 Base'] - data['Employment 2017 Base'])/data['Employment 2017 Base'])*100

data['Difference 2023-BEA Base'] = data['Employment 2023 Base'] - data['Employment BEA']
data['Difference % 2023-BEA Base'] = ((data['Employment 2023 Base'] - data['Employment BEA'])/data['Employment BEA'])*100

In [42]:
data['Year'].unique()

array(['2017', '2020', '2025', '2035', '2045', '2018', '2019', '2021',
       '2022'], dtype=object)

In [43]:
data.tail()

,NAME,Year,Employment 2023 Base,Employment 2017 Base,Employment BEA,Difference 2023-2017 Base,Difference % 2023-2017 Base,Difference 2023-BEA Base,Difference % 2023-BEA Base
148,"Williamson County, Tennessee",2022,NaN,NaN,238165.0,NaN,NaN,NaN,NaN
149,"Wilson County, Tennessee",2022,NaN,NaN,94211.0,NaN,NaN,NaN,NaN
150,MPO,2022,NaN,NaN,1480965.0,NaN,NaN,NaN,NaN
151,GNRC,2022,NaN,NaN,1583744.0,NaN,NaN,NaN,NaN
152,GNRC Region,2022,NaN,NaN,1643234.0,NaN,NaN,NaN,NaN


In [44]:
data.to_csv('../data/totalemployment_projectioncomps.csv', index = False)